## Named Entity Recognition

In [11]:
#imports 
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch

In [6]:
model_id = "dslim/bert-base-NER"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id) #huggingface model

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
ner_model = AutoModelForTokenClassification.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

In [29]:
#nlp pipeline
nlp = pipeline('ner', model=ner_model, tokenizer=tokenizer, aggregation_strategy = 'max', device = None)

In [20]:
text = "“We formed our partnership with OpenAI around a shared ambition to responsibly advance cutting-edge AI \
    research and democratize AI as a new technology platform,” said Satya Nadella, Chairman and CEO, Microsoft.\
          “In this next phase of our partnership, developers and organizations across industries will have access\
              to the best AI infrastructure, models, and toolchain with Azure to build and run their applications.”"

In [21]:
nlp(text)

[{'entity_group': 'ORG',
  'score': 0.99865675,
  'word': 'OpenAI',
  'start': 32,
  'end': 38},
 {'entity_group': 'MISC',
  'score': 0.9881143,
  'word': 'AI',
  'start': 100,
  'end': 102},
 {'entity_group': 'MISC',
  'score': 0.9725788,
  'word': 'AI',
  'start': 132,
  'end': 134},
 {'entity_group': 'PER',
  'score': 0.99965036,
  'word': 'Satya Nadella',
  'start': 171,
  'end': 184},
 {'entity_group': 'ORG',
  'score': 0.9988427,
  'word': 'Microsoft',
  'start': 204,
  'end': 213},
 {'entity_group': 'MISC',
  'score': 0.9932892,
  'word': 'AI',
  'start': 353,
  'end': 355},
 {'entity_group': 'ORG',
  'score': 0.9938997,
  'word': 'Azure',
  'start': 399,
  'end': 404}]

Clean Pinecone
1. Clean pinecone index
2. Init pinecone
3. Delete data from index
4. Load libraries for NER
5. Load libraries for retriever
6. Upsert data
7. Query data

In [25]:
api_key = "key"
env = "env"

In [28]:
from pinecone import Pinecone, PodSpec
from tqdm.autonotebook import tqdm
pc = Pinecone(api_key=api_key)
#delete data in old index (delete the actual index in the Pinecone client)
idx = pc.Index('medium-data')
idx.delete(delete_all=True)

{}

In [ ]:
#nlp pipeline
nlp = pipeline('ner', model=ner_model, tokenizer=tokenizer, aggregation_strategy = 'max', device = 'cpu')

In [30]:
nlp("Bill Gates is the founder of Microsoft") #test the changed nlp using cpu as engine

[{'entity_group': 'PER',
  'score': 0.9997382,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.99829453,
  'word': 'Microsoft',
  'start': 29,
  'end': 38}]

In [32]:
from sentence_transformers import SentenceTransformer
#https://huggingface.co/flax-sentence-embeddings/all_datasets_v3_mpnet-base
#NOTE: Vector size is 768 in dimension
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base")

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [33]:
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [34]:
pc.create_index(name='medium-data',dimension=768,metric='cosine',spec=PodSpec(
    environment=env,
    pod_type= "p1.x1",
    pods= 1
  ))
idx = pc.Index('medium-data')

In [39]:
from datasets import load_dataset
import pandas as pd

In [45]:
df = load_dataset("fabiochiu/medium-articles", data_files="medium_articles.csv", split="train").to_pandas()

Generating train split: 0 examples [00:00, ? examples/s]

## Data preparation 

In [112]:
df_short = df.dropna().sample(100, random_state=45) #10k can take more than 1h when embedding & upserting

In [113]:
df_short['text_extended'] = df_short['title'] + '.' + df_short['text'].str[:1000]

In [114]:
batch = df_short['text_extended'].iloc[0:10].tolist()

In [115]:
#[item['word'] for item in nlp(batch)] #list of entities for 1 document when batch = iloc[0]
len(nlp(batch))

10

## Helper function

In [116]:
#helper function for entity extraction
def extract_entities(batch):
    entities = []
    for doc in batch:
        entities.append([item['word'] for item in nlp(doc)])
    return entities

In [117]:
len(retriever.encode(batch[0])) #note 768, same as our dimensions in Pinecone

768

In [118]:
#embedding for batch
#emb = retriever.encode(batch).tolist()

### Main script, embedding, creating metadata, upserting to Pinecone

In [123]:
from tqdm.auto import tqdm
batch_size = 64
for i in range(0, len(df_short), batch_size):
    #start and end index of each batch
    i_end = min(i+batch_size, len(df_short))
    print(i, i_end) #keep track of the progress
    #get a batch of data
    batch = df_short.iloc[i:i_end]
    #embedding
    emb = retriever.encode(batch['text_extended'].tolist()).tolist()
    #NER extraction
    entities = extract_entities(batch['text_extended'].tolist())
    #[[]] --> [set1, set2, ...]
    batch['named_entity'] = [list(set(entity)) for entity in entities] 
    #removing duplicates, one list per document

    #create metadata
    batch = batch.drop('text', axis=1)
    metadata = batch.to_dict(orient='records')

    ids = [f"{index}" for index in range(i, i_end)]
    vectors_upsert = list(zip(ids, emb, metadata))
    _ = idx.upsert(vectors=vectors_upsert)


C:\Users\rbystrom\AppData\Local\Temp\ipykernel_4128\1559257549.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['named_entity'] = [list(set(entity)) for entity in entities]
C:\Users\rbystrom\AppData\Local\Temp\ipykernel_4128\1559257549.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['named_entity'] = [list(set(entity)) for entity in entities]
C:\Users\rbystrom\AppData\Local\Temp\ipykernel_4128\1559257549.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

### Query data

In [143]:
query = "Wordpress" #Natural Language
qx = retriever.encode(query).tolist() #Query vector (embedded)
ne = extract_entities([query])[0] #Named entity as search filter

In [146]:
xc = idx.query(vector=qx, top_k=5,include_metadata=True,filter={"named_entity": {"$in" : ne}})

In [147]:
for result in xc['matches']:
    print(result['score'], ' ', result['metadata']['named_entity'])

0.244298488   ['Unsplash', 'Moritz Mentges', 'WordPress', 'Linux Hosts Inc']
0.234006554   []
0.232506484   []
0.226189375   ['Wyze Cam', 'Ring Alarm', 'Noonlight', 'TechHive', 'Wyze Labs', 'Ring', 'Wyze']
0.211620227   ['Guarda Wallet', 'Waves Platform', 'Waves', 'Wallet', 'Guarda']


Better results when the whole dataset is imported